In [11]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path




In [26]:
loader = PyPDFLoader("ScrumGuideEng.pdf")
documents = loader.load_and_split()
len(documents) # 2

15

In [27]:
print(documents[5].page_content)

4  
Transparency enables inspection. Inspection without transparency is misleading and wasteful.  
Inspection  
The Scrum artifacts and the progress toward agreed goals must be inspected frequently and diligently to 
detect potentially undesirable variances or problems. To help with inspection, Scrum provides cadence 
in the form of its five events.  
  
Inspecti on enables adaptation. Inspection without adaptation is considered pointless. Scrum events are 
designed to provoke change.  
Adaptation  
If any aspects of a process deviate outside acceptable limits or if the resulting product is unacceptable, 
the process bei ng applied or the materials being produced must be adjusted. The adjustment must be 
made as soon as possible to minimize further deviation.  
  
Adaptation becomes more difficult when the people involved are not empowered or self -managing. A 
Scrum Team is exp ected to adapt the moment it learns anything new through inspection.  
Scrum Values  
Successful use of Scru

In [28]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=64
)
texts = text_splitter.split_documents(documents)
len(texts) # 3

41

In [29]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [30]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

In [31]:
local_path = ("./models/orca-mini-3b.ggmlv3.q4_0.bin") 

In [32]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [33]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
model = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

Found model file at  ./models/orca-mini-3b.ggmlv3.q4_0.bin
Found model file at  ./models/orca-mini-3b.ggmlv3.q4_0.bin


In [34]:
qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False,
)

In [36]:
res = qa(f"""
what is Scrum Master?
 Extract the Answer from text.
""")


 
Scrum Masters are accountable for establishing Scrum as defined in the Scrum Guide. They do this by helping everyone understand Scrum theory and practice, both within the Scrum Team and the organization. The Scrum Master is a true leader who serves the Scrum Team and the larger organization. They serve the Scrum Team in several ways, including coaching the team members in self-management and cross-functionality; causing the removal of impediments to the Scrum Team’s progress; and ensuring that all Scrum events take place and are positive, productive, and kept within the timebox. The Scrum Master serves the organization in several ways, including leading, training, and coaching the organization in its Scrum adoption. Planning and advising Scrum implementations within the organization. Helping employees and stakeholders understand and enact an empirical approach for complex work. Removing barriers between stakeholders and Scrum Teams.